In [ ]:
!pip3 install geojson
!pip3 install osmnx
!pip3 install shapely.constructive
import pandas as pd
import numpy as np
import csv
import plotly.express as px
import geopandas as gpd
import json
import requests
# import ipython-sql
# import regex as re
%matplotlib inline
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 2.0 MB/s 
     |████████████████████████████████| 1.0 MB 8.6 MB/s 
     |████████████████████████████████| 1.0 MB 57.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 6.3 MB 53.1 MB/s 
     |████████████████████████████████| 11.2 MB 31.4 MB/s 
     |████████████████████████████████| 16.7 MB 329 kB/s 
     |████████████████████████████████| 944 kB 50.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shapely.constructive (from versions: none)
ERROR: No matching distribution found for shapely.constructive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
workdir = '/content/drive/MyDrive/Colab Notebooks/Hazus Data/' # for Tina
# workdir = '/content/drive/MyDrive/Hazus Data/'                 # for Adam

Mounted at /content/drive


In [ ]:
def read_tables(path,state):
  df = pd.read_csv(path,sep =',')
  df['State'] = state
  cols = list(df)
  cols = [cols[-1]] + cols[:-1]
  cols = df[cols]
  return cols

In [ ]:
state = 'NY'
path_NY = f'{workdir}Raw_Data/{state}/NY_hzBldgCountOccupB.csv'
NY_hzBldgCountOccupB = read_tables(path_NY,state)

In [ ]:
table = NY_hzBldgCountOccupB.copy()

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_hzSqFootageOccupB.csv'
NY_hzSqFootageOccupB = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_hzDemographicsB.csv'
NY_hzDemographicsB = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_flNightVehicleInv.csv'
NY_flNightVehicleInv = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_flDayVehicleInv.csv'
NY_flDayVehicleInv = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_hzExposureOccupB.csv'
NY_ExposureOccupB = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_hzExposureContentOccupB.csv'
NY_ExposureContentOccupB = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_hzCensusBlock_TIGER.csv'
NY_CensusBlock_TIGER = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_huCensusBlock.csv'
NY_huCensusBlock = pd.read_csv(path)

In [ ]:
path = f'{workdir}Raw_Data/{state}/NY_huTreeParametersB.csv'
NY_huTreeParametersB = pd.read_csv(path)

In [ ]:
US_FIPS = pd.read_csv(f'{workdir}US_FIPS.csv')
NY_FIPS = US_FIPS.loc[US_FIPS["state"] == "NY"]

In [ ]:
NY_FIPS

,fips,name,state
1862,36001,Albany County,NY
1863,36003,Allegany County,NY
1864,36005,Bronx County,NY
1865,36007,Broome County,NY
1866,36009,Cattaraugus County,NY
...,...,...,...
1919,36115,Washington County,NY
1920,36117,Wayne County,NY
1921,36119,Westchester County,NY
1922,36121,Wyoming County,NY


In [ ]:
def find_code(name):
  row = US_FIPS.loc[US_FIPS["name"].str.contains(name)]
  row = row["fips"].values[0]
  return row

In [ ]:
# for i in NY_FIPS["fips"]:
#   row = NY_FIPS.loc[NY_FIPS["fips"] == i].values[0]
#   code = row[0]
#   name = row[1]
#   table["County Code"] = NY_hzBldgCountOccupB["CensusBlock"].astype(str).str[:5].astype(int)
#   table["CensusBlock"].astype(int)
#   table = table.loc[table['County Code'] == code]
#   NY_sum = NY_hzSqFootageOccupB.merge(table, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_flNightVehicleInv, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_flDayVehicleInv, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_ExposureOccupB, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_ExposureContentOccupB, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_huTreeParametersB, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_huTerrainB, how = "inner", on = "CensusBlock")
#   NY_sum = NY_sum.merge(NY_hzDemographicsB,how = "inner", on = "CensusBlock")
#   table = NY_hzBldgCountOccupB.copy()
#   NY_sum.to_csv(f'{workdir}{code}_{name}_sum.csv')

In [ ]:
# name = 'Albany'
# code = find_code(name)
# county_table = pd.read_csv(f'{workdir}Hazus_NY_CB/{code}_{name} County_sum.csv').drop(columns = "Unnamed: 0")
# county_table

In [ ]:
# create a copy (just to make sure I do not mess with the original file; this is not necessary when you apply the method)
NY_hzBldgCountOccupB_CB = NY_hzBldgCountOccupB.copy()

# set CensusBlock to int64
NY_hzBldgCountOccupB_CB["County Code"] = NY_hzBldgCountOccupB_CB["CensusBlock"].astype(str).str[:5].astype(int)
NY_hzBldgCountOccupB_CB['CensusBlock'] = NY_hzBldgCountOccupB_CB['CensusBlock'].astype(np.int64)

# set index to CensusBlock
NY_hzBldgCountOccupB_CB.set_index('CensusBlock', inplace=True)

# Remove ObjectID
NY_hzBldgCountOccupB_CB.drop('OBJECTID', axis=1, inplace=True)

# Remove the State column as it is redundant information that takes up a lot of space
NY_hzBldgCountOccupB_CB.drop('State', axis=1, inplace=True)

# 'I' (the last character of occupancy types) does not have additional information;
# it is just there to have unique names for these columns when joined with columns from other tables. Let's remove it.
cols = [col[:-1] if col.endswith('I') else col for col in NY_hzBldgCountOccupB_CB.columns]
NY_hzBldgCountOccupB_CB.columns = cols

# Now create a MultiIndex Header
MI = pd.MultiIndex.from_product([['BuildingCount'],NY_hzBldgCountOccupB_CB.columns], names=['group','feature'])
NY_hzBldgCountOccupB_CB.columns = MI

# BuildingCount table DONE

# create a copy of the next table (again, just to keep the example clean)
NY_hzSqFootageOccupB_CB = NY_hzSqFootageOccupB.copy()

# same thing with the CensusBlock column going to index, ObjectID removed
# perhaps you could add some of these things - and remove the State column in the read_tables() method
NY_hzSqFootageOccupB_CB['CensusBlock'] = NY_hzSqFootageOccupB_CB['CensusBlock'].astype(np.int64)
NY_hzSqFootageOccupB_CB.set_index('CensusBlock', inplace=True)
NY_hzSqFootageOccupB_CB.drop('OBJECTID', axis=1, inplace=True)

# now remove the 'F' from the column names
cols = [col[:-1] if col.endswith('F') else col for col in NY_hzSqFootageOccupB_CB.columns]
NY_hzSqFootageOccupB_CB.columns = cols

# It also doesn't make much sense to store the values in "Thousand sq. ft", I suggest storing them in "sq. ft" units
# Note that floating point numbers take the same amount of space regardless of the size of the actual number
# So, we multiply everything in this table by 1000
NY_hzSqFootageOccupB_CB *= 1000

# and create a MultiIndex header
MI = pd.MultiIndex.from_product([['TotalSquareFootage'],NY_hzSqFootageOccupB_CB.columns], names=['group','feature'])
NY_hzSqFootageOccupB_CB.columns = MI

# TotalSquareFootage table DONE

# I'll do one more for the sake of the example
NY_flNightVehicleInv_CB = NY_flNightVehicleInv.copy()

# same preparation steps as before
NY_flNightVehicleInv_CB['CensusBlock'] = NY_flNightVehicleInv_CB['CensusBlock'].astype(np.int64)
NY_flNightVehicleInv_CB.set_index('CensusBlock', inplace=True)
NY_flNightVehicleInv_CB.drop('OBJECTID', axis=1, inplace=True)

# Column names are okay here, I suggest keeping them as-is

# Multiply exposure values with 1000 to convert "Thousand $" units with "$"
# See the CDMS Data dictionary for the units of each column
cols = []
for col in NY_flNightVehicleInv_CB.columns:
  if col.startswith('TVal'):
    cols.append(col)
cols.append('TotalExp')
NY_flNightVehicleInv_CB[cols] *= 1000

# Add the MultiIndex header
MI = pd.MultiIndex.from_product([['VehiclesNight'],NY_flNightVehicleInv_CB.columns], names=['group','feature'])
NY_flNightVehicleInv_CB.columns = MI

# VehiclesNight table DONE

# Here you'll need to prepare the other tables in similar fashion

# VehiclesDay / NY_flDayVehicleInv
# same idea as the night vehicles

NY_flDayVehicleInv_CB = NY_flDayVehicleInv.copy()
NY_flDayVehicleInv_CB['CensusBlock'] = NY_flDayVehicleInv_CB['CensusBlock'].astype(np.int64)
NY_flDayVehicleInv_CB.set_index('CensusBlock', inplace=True)
NY_flDayVehicleInv_CB.drop('OBJECTID', axis=1, inplace=True)

cols = []
for col in NY_flDayVehicleInv_CB.columns:
  if col.startswith('TVal'):
    cols.append(col)
cols.append('TotalExp')
NY_flDayVehicleInv_CB[cols] *= 1000

MI = pd.MultiIndex.from_product([['VehiclesDay'],NY_flDayVehicleInv_CB.columns], names=['group','feature'])
NY_flDayVehicleInv_CB.columns = MI

# VehiclesDay table DONE

# ReplacementValue / NY_ExposureOccupB
# Values are in "Thousand $" and "I" needs to be removed from occupancy type header labels
# Very similar to the square footage table

NY_ExposureOccupB_CB = NY_ExposureOccupB.copy()
NY_ExposureOccupB_CB['CensusBlock'] = NY_ExposureOccupB_CB['CensusBlock'].astype(np.int64)
NY_ExposureOccupB_CB.set_index('CensusBlock', inplace=True)
NY_ExposureOccupB_CB.drop('OBJECTID', axis=1, inplace=True)

cols = [col[:-1] if col.endswith('I') else col for col in NY_ExposureOccupB_CB.columns]
NY_ExposureOccupB_CB.columns = cols
NY_ExposureOccupB_CB *= 1000

MI = pd.MultiIndex.from_product([['ReplacementValue'],NY_ExposureOccupB_CB.columns], names=['group','feature'])
NY_ExposureOccupB_CB.columns = MI

# ContentValue / NY_ExposureContentOccupB
# Same treatment as the replacement value

NY_ExposureContentOccupB_CB = NY_ExposureContentOccupB.copy()
NY_ExposureContentOccupB_CB['CensusBlock'] = NY_ExposureContentOccupB_CB['CensusBlock'].astype(np.int64)
NY_ExposureContentOccupB_CB.set_index('CensusBlock', inplace=True)
NY_ExposureContentOccupB_CB.drop('OBJECTID', axis=1, inplace=True)

cols = [col[:-1] if col.endswith('I') else col for col in NY_ExposureContentOccupB_CB.columns]
NY_ExposureContentOccupB_CB.columns = cols
NY_ExposureContentOccupB_CB *= 1000

MI = pd.MultiIndex.from_product([['ContentValue'],NY_ExposureContentOccupB_CB.columns], names=['group','feature'])
NY_ExposureContentOccupB_CB.columns = MI

# Demographics / NY_hzDemographicsB
# I would not change the header labels and there is nothing to convert, so this is a simple one

NY_hzDemographicsB_CB = NY_hzDemographicsB.copy()
NY_hzDemographicsB_CB['CensusBlock'] = NY_hzDemographicsB_CB['CensusBlock'].astype(np.int64)
NY_hzDemographicsB_CB.set_index('CensusBlock', inplace=True)
NY_hzDemographicsB_CB.drop('OBJECTID', axis=1, inplace=True)
MI = pd.MultiIndex.from_product([['Demographics'],NY_hzDemographicsB_CB.columns], names=['group','feature'])
NY_hzDemographicsB_CB.columns = MI

# Tiger/NY_CensusBlock_TIGER
NY_CensusBlock_TIGER_CB = NY_CensusBlock_TIGER.copy()
NY_CensusBlock_TIGER_CB['CensusBlock'] = NY_CensusBlock_TIGER_CB['CensusBlock'].astype(np.int64)
NY_CensusBlock_TIGER_CB.set_index('CensusBlock', inplace=True)
NY_CensusBlock_TIGER_CB.drop(['OBJECTID','Tract', 'BlockArea','CenLat','CenLongit','IncomeRatio'],axis=1, inplace=True)
NY_CensusBlock_TIGER_CB.drop('Shape', axis=1, inplace=True)
for cols in NY_CensusBlock_TIGER_CB.iloc[:,2:]:
    NY_CensusBlock_TIGER_CB[cols] = NY_CensusBlock_TIGER_CB[cols].astype(float)*0.01

MI = pd.MultiIndex.from_product([['TIGERCensusBlock'],NY_CensusBlock_TIGER_CB.columns], names=['group','feature'])
NY_CensusBlock_TIGER_CB.columns = MI


# TreeParameter Table
NY_huTreeParametersB_CB = NY_huTreeParametersB.copy()
NY_huTreeParametersB_CB['CensusBlock'] = NY_huTreeParametersB_CB['CensusBlock'].astype(np.int64)
NY_huTreeParametersB_CB.set_index('CensusBlock', inplace=True)
NY_huTreeParametersB_CB.drop('OBJECTID',axis=1, inplace=True)
for cols in NY_huTreeParametersB_CB.loc[:,'TreeHeightLess40':'TreeHeightGreater60']:
    NY_huTreeParametersB_CB[cols] = NY_huTreeParametersB_CB[cols].astype(float)*0.01

MI = pd.MultiIndex.from_product([['TreeParameter'],NY_huTreeParametersB_CB.columns], names=['group','feature'])
NY_huTreeParametersB_CB.columns = MI


# Hurricane Census Block Table
NY_huCensusBlock_CB = NY_huCensusBlock.copy()
NY_huCensusBlock_CB['CensusBlock'] = NY_huCensusBlock_CB['CensusBlock'].astype(np.int64)
NY_huCensusBlock_CB.set_index('CensusBlock', inplace=True)
NY_huCensusBlock_CB.drop('OBJECTID',axis=1, inplace=True)

MI = pd.MultiIndex.from_product([['Hurricane'],NY_huCensusBlock_CB.columns], names=['group','feature'])
NY_huCensusBlock_CB.columns = MI

In [ ]:
# And in the end, just concatenate everything
SummaryTable = pd.concat(
    [NY_hzBldgCountOccupB_CB,
     NY_hzSqFootageOccupB_CB,
     NY_flNightVehicleInv_CB,
     NY_flDayVehicleInv_CB,
     NY_ExposureOccupB_CB,
     NY_ExposureContentOccupB_CB,
     NY_hzDemographicsB_CB,
     NY_CensusBlock_TIGER_CB,
     NY_huTreeParametersB_CB,
     NY_huCensusBlock_CB],
     axis=1)

# save the file using the naming convention you had before, but do not add _sum in the end
# all of our talbes are going to be sums, so that does not really add new information for the user

SummaryTable

group           BuildingCount                                                \
feature                  RES1 RES2 RES3A RES3B RES3C RES3D RES3E RES3F RES4   
CensusBlock                                                                   
360010001001000           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
360010001001001           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
360010001001002           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
360010001001003           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
360010001001004           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
...                       ...  ...   ...   ...   ...   ...   ...   ...  ...   
361231505004022           2.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
361231505004023           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
361231505004024           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
361231505004025           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   
361231505004026           0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0  0.0   

group                 ... TreeParameter                                    \
feature         RES5  ...  StemsPerAcre TreeHeightLess40 TreeHeight40To60   
CensusBlock           ...                                                   
360010001001000  0.0  ...             9             0.12             0.55   
360010001001001  0.0  ...            92             0.12             0.55   
360010001001002  0.0  ...            25             0.12             0.55   
360010001001003  0.0  ...            19             0.12             0.55   
360010001001004  0.0  ...            30             0.12             0.55   
...              ...  ...           ...              ...              ...   
361231505004022  0.0  ...             4             0.03             0.58   
361231505004023  0.0  ...            17             0.03             0.58   
361231505004024  0.0  ...             0             0.03             0.58   
361231505004025  0.0  ...             0             0.03             0.58   
361231505004026  0.0  ...             0             0.03             0.58   

group                                                            Hurricane  \
feature         TreeHeightGreater60 TreeCollectionFactor  huBldgSchemeName   
CensusBlock                                                                  
360010001001000                0.33              0.17682  Northeast_Inland   
360010001001001                0.33              0.08318  Northeast_Inland   
360010001001002                0.33              1.00000  Northeast_Inland   
360010001001003                0.33              0.48819  Northeast_Inland   
360010001001004                0.33              0.10012  Northeast_Inland   
...                             ...                  ...               ...   
361231505004022                0.39              0.37937  Northeast_Inland   
361231505004023                0.39              0.82660  Northeast_Inland   
361231505004024                0.39              0.51904  Northeast_Inland   
361231505004025                0.39              0.73768  Northeast_Inland   
361231505004026                0.39              0.47352  Northeast_Inland   

group                                                                          
feature         huOccMapSchemeName DistToCoast SURFACEROUGHNESS WindGridIndex  
CensusBlock                                                                    
360010001001000   Northeast_Inland  166.300003            0.512             1  
360010001001001   Northeast_Inland  166.300003            0.496             1  
360010001001002   Northeast_Inland  166.300003            0.506             1  
360010001001003   Northeast_Inland  166.300003            0.499             1  
360010001001004   Northeast_Inland  166.300003            0.638             1  
...                            ...         ...              ...           ...  
3

In [ ]:
for i in NY_FIPS["fips"]:
  row = NY_FIPS.loc[NY_FIPS["fips"] == i].values[0]
  code = row[0]
  name = row[1]
  table = SummaryTable.copy()
  NY_sum = table.loc[table[('BuildingCount','County Code')] == code]
  table = SummaryTable.copy()
  NY_sum.to_csv(f'{workdir}Processed_Data/NY/NY_{code}_{name}_sum.csv')

In [ ]:
for cols in SummaryTable.columns:
  print(cols)

('BuildingCount', 'RES1')
('BuildingCount', 'RES2')
('BuildingCount', 'RES3A')
('BuildingCount', 'RES3B')
('BuildingCount', 'RES3C')
('BuildingCount', 'RES3D')
('BuildingCount', 'RES3E')
('BuildingCount', 'RES3F')
('BuildingCount', 'RES4')
('BuildingCount', 'RES5')
('BuildingCount', 'RES6')
('BuildingCount', 'COM1')
('BuildingCount', 'COM2')
('BuildingCount', 'COM3')
('BuildingCount', 'COM4')
('BuildingCount', 'COM5')
('BuildingCount', 'COM6')
('BuildingCount', 'COM7')
('BuildingCount', 'COM8')
('BuildingCount', 'COM9')
('BuildingCount', 'COM10')
('BuildingCount', 'IND1')
('BuildingCount', 'IND2')
('BuildingCount', 'IND3')
('BuildingCount', 'IND4')
('BuildingCount', 'IND5')
('BuildingCount', 'IND6')
('BuildingCount', 'AGR1')
('BuildingCount', 'REL1')
('BuildingCount', 'GOV1')
('BuildingCount', 'GOV2')
('BuildingCount', 'EDU1')
('BuildingCount', 'EDU2')
('BuildingCount', 'County Code')
('TotalSquareFootage', 'RES1')
('TotalSquareFootage', 'RES2')
('TotalSquareFootage', 'RES3A')
('TotalS